In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Параметры колебательного звена
T = 10 # Уменьшение T приведет к уменьшению периода колебаний и времени переходного процесса
xi = 0.02 #Увеличение приведет к уменьшению перерегулирования и времени регулирования, но увеличит затухание колебаний
K = 1 # Влияет на амплитуду колебаний

omega_n = 1 / T
omega_d = omega_n * np.sqrt(1 - xi**2)

t_max = 2000
t = np.linspace(0, t_max, 1500)

# Аналитическое решение
def h_analytical(t):
    return K * (1 - np.exp(-xi * omega_n * t) * \
                (np.cos(omega_d * t) + (xi / np.sqrt(1 - xi**2)) * np.sin(omega_d * t)))

# Численное решение
def system(t, y):
    dydt = [
        y[1],
        -(1 / T**2) * y[0] - (2 * xi / T) * y[1] + (K / T**2)
    ]
    return dydt

y0 = [0, 0]
solution = solve_ivp(system, [0, t_max], y0, t_eval=t, method='RK45')
y_numerical = solution.y[0]

# Построение графика аналитического и численного решений
plt.figure(figsize=(10, 6))
plt.plot(t, h_analytical(t), 'b-', label='Аналитическое решение')
plt.plot(t, y_numerical, 'r--', label='Численное решение')

plt.axhline(y=1.05, color='green', linestyle='--', label='y = 1.05')
plt.axhline(y=0.95, color='purple', linestyle='--', label='y = 0.95')

# Настройки графика
plt.grid(True)
plt.xlabel('Время, с')
plt.ylabel('Выходная величина')
plt.title('Переходная характеристика колебательного звена')
plt.legend()
plt.show()

# Расчет прямых показателей качества
max_value = np.max(h_analytical(t))
overshoot = (max_value - K) / K * 100
print(f"Перерегулирование: {overshoot:.2f}%")

max_idx = np.argmax(h_analytical(t))
t_max_response = t[max_idx]
print(f"Время первого максимума: {t_max_response:.2f} с")

T_osc = 2 * np.pi / omega_d
print(f"Период колебаний: {T_osc:.2f} с")

# Расчет времени регулирования 5%
settling_band = 0.05 * K
steady_state = K
in_band = np.abs(h_analytical(t) - steady_state) <= settling_band
change_points = np.diff(in_band.astype(int))
if np.any(change_points):
    last_exit = np.max(np.where(change_points != 0)[0])
    t_settling = t[last_exit + 1]
else:
    t_settling = 0
print(f"Время регулирования (5%): {t_settling:.2f} с")

# Теоретическое перерегулирование
theoretical_overshoot = np.exp(-np.pi * xi / np.sqrt(1 - xi**2)) * 100
print(f"Теоретическое перерегулирование: {theoretical_overshoot:.2f}%")

# Теоретическое время переходного процесса
t_settle_theor = 3 / (xi * omega_n)
print(f"Теоретическое время переходного процесса: {t_settle_theor:.2f} с")

# Число колебаний за время регулирования
n_osc = t_settling / T_osc
print(f"Число колебаний за время регулирования: {n_osc:.2f}")


In [ ]:
from scipy.signal import lti, freqresp, step

T = 1
xi = 0.2
K = 10

print(f"Параметры: T = {T}, ξ = {xi}, K = {K}")

num_ol = [K]
den_ol = [T**2, 2*xi*T, 1]
open_loop_system = lti(num_ol, den_ol)
print(f"Разомкнутая ветвь: W(s) = {K} / ({T**2}·s² + {2*xi*T}·s + 1)")

# Передаточная функция замкнутой системы (OTU: Φ(s) = W(s)/(1+W(s)))
closed_loop_den = np.polyadd(den_ol, num_ol)
closed_loop_system = lti(num_ol, closed_loop_den)
print(f"Замкнутая система: Φ(s) = {num_ol[0]} / ({closed_loop_den[0]}·s² + {closed_loop_den[1]}·s + {closed_loop_den[2]})")

# Частотный анализ для разомкнутой системы
# Расчёт АЧХ и ФЧХ через freqresp

omega = np.logspace(-2, 2, 1000)
w, h = freqresp(open_loop_system, omega)
mag = np.abs(h)
phase = np.angle(h, deg=True)

# График АЧХ и ФЧХ разомкнутой системы
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.semilogx(w, mag, 'b-', label='Амплитуда')
ax1.set_xlabel('Частота (рад/с)')
ax1.set_ylabel('Амплитуда', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True, which="both", ls="--")
ax2 = ax1.twinx()
ax2.semilogx(w, phase, 'r-', label='Фаза')
ax2.set_ylabel('Фаза (°)')
ax2.tick_params(axis='y', labelcolor='r')
plt.title('АЧХ и ФЧХ разомкнутой системы')
plt.tight_layout()
plt.show()

# Фазовый запас: находим частоту, где |H(jω)|≈1, затем PM = 180 + фаза
unity_idx = np.argmin(np.abs(mag - 1))
phase_margin = 180 + phase[unity_idx]
crossover_freq = w[unity_idx]

# Амплитудный запас: ищем частоту, при которой фаза ≈ -180°
phase_180_indices = np.where(np.abs(phase + 180) < 1)[0]
if len(phase_180_indices) > 0:
    idx = phase_180_indices[0]
    gain_margin = 1 / mag[idx]
    gain_margin_db = 20 * np.log10(gain_margin)
    phase_crossover_freq = w[idx]
else:
    gain_margin = gain_margin_db = phase_crossover_freq = None

# Частотный анализ замкнутой системы – расчет АЧХ и ФЧХ
w_cl, h_cl = freqresp(closed_loop_system, omega)
mag_cl = np.abs(h_cl)
phase_cl = np.angle(h_cl, deg=True)

# Находим резонансный пик и его частоту
M = np.max(mag_cl)
M_idx = np.argmax(mag_cl)
resonance_freq = w_cl[M_idx]

# Определяем полосу пропускания
bandwidth_indices = np.where(20 * np.log10(mag_cl) <= -3)[0]
if len(bandwidth_indices) > 0:
    bandwidth_idx = bandwidth_indices[0]
    bandwidth = w_cl[bandwidth_idx]
else:
    bandwidth = None

# График АЧХ и ФЧХ замкнутой системы
fig, ax1 = plt.subplots(figsize=(10, 6))
ax1.semilogx(w_cl, mag_cl, 'b-',  label='Амплитуда (Замкнутая)')
ax1.set_xlabel('Частота (рад/с)')
ax1.set_ylabel('Амплитуда', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True, which="both", ls="--")
ax2 = ax1.twinx()
ax2.semilogx(w_cl, phase_cl, 'r-',  label='Фаза (Замкнутая)')
ax2.set_ylabel('Фаза (°)', )
ax2.tick_params(axis='y', labelcolor='r')
plt.title('АЧХ и ФЧХ замкнутой системы')
plt.tight_layout()
plt.show()

# Переходная характеристика замкнутой системы
t = np.linspace(0, 20, 1000)
t_out, y_step = closed_loop_system.step(T=t)

plt.figure(figsize=(10, 6))
plt.plot(t_out, y_step, 'b-',  label='Переходная характеристика')
steady_state_value = y_step[-1]
plt.axhline(y=steady_state_value * 1.05, color='green', linestyle='--', label='y = 1.05')
plt.axhline(y=steady_state_value * 0.95, color='purple', linestyle='--', label='y = 0.95')
plt.xlabel('Время (с)')
plt.ylabel('Амплитуда')
plt.title('Переходная характеристика замкнутой системы')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


omega_n_cl = np.sqrt(1 + K)
zeta_cl = xi / omega_n_cl
omega_d_cl = omega_n_cl * np.sqrt(1 - zeta_cl**2)

# Теоретическое перерегулирование (%)
Mp_theory = np.exp(-np.pi * zeta_cl / np.sqrt(1 - zeta_cl**2)) * 100

# Теоретическое время регулирования (5%-критерий)
Ts_theory = 3 / (zeta_cl * omega_n_cl)

# Период колебаний
T_osc = 2 * np.pi / omega_d_cl

# Вывод показателей качества
print("\nКосвенные показатели качества замкнутой системы:")
print("--------------------------------------------------")
print(f"Фазовый запас (по разомкнутой ветви): {phase_margin:.2f}°")
print(f"Частота среза (фазовый переход): {crossover_freq:.2f} рад/с")
if gain_margin is not None:
    print(f"Амплитудный запас: {gain_margin_db:.2f} дБ при частоте {phase_crossover_freq:.2f} рад/с")
else:
    print("Амплитудный запас: не определён (фаза не достигает -180°)")
print(f"Показатель колебательности (резонансный пик, M): {M:.2f}")
print(f"Резонансная частота: {resonance_freq:.2f} рад/с")
if bandwidth is not None:
    print(f"Полоса пропускания: {bandwidth:.2f} рад/с")
else:
    print("Полоса пропускания: не определена")

settling_idx = np.where(np.abs(y_step - steady_state_value) <= 0.05 * steady_state_value)[0]
settling_time_empirical = t_out[settling_idx[0]] if len(settling_idx) > 0 else t_out[-1]
print(f"Эмпирическое время регулирования (5%): {settling_time_empirical:.2f} с")
print(f"Теоретическое время регулирования: {Ts_theory:.2f} с")
print(f"Период колебаний: {T_osc:.2f} с")
print(f"Теоретическое перерегулирование: {Mp_theory:.2f} %")

# Теоретическое время переходного процесса по закону 3/(ξ·ωn) для разомкнутой системы
t_p_theory = 3 / (xi * (1/T))
print(f"Теоретическое время переходного процесса (разомкнутой): {t_p_theory:.2f} с")


1. Фазовый запас (по разомкнутой ветви): 7.60°
Описание: Фазовый запас — это разница между фазой разомкнутой системы и критической фазой (-180°) при частоте, на которой амплитудный коэффициент равен 1 (0 дБ). Это показывает, насколько далеко система от границы нестабильности.
Влияние: Больший фазовый запас указывает на большую стабильность системы. Низкий фазовый запас может привести к нестабильности и колебаниям.
2. Частота среза (фазовый переход): 3.30 рад/с
Описание: Частота, на которой фаза разомкнутой системы достигает критического значения (-180°), часто совпадает с частотой, где амплитудный коэффициент равен 1.
Влияние: Эта частота показывает, где система начинает терять стабильность при увеличении частоты.
3. Амплитудный запас: 34.52 дБ при частоте 23.09 рад/с
Описание: Амплитудный запас — это разница между амплитудным коэффициентом разомкнутой системы и критическим значением (0 дБ) при частоте, где фаза равна -180°. Измеряется в децибелах (дБ).
Влияние: Больший амплитудный запас указывает на большую стабильность и запас по амплитуде. Низкий амплитудный запас может привести к нестабильности.
4. Показатель колебательности (резонансный пик, M): 7.55
Описание: Резонансный пик — это максимальное значение амплитудного коэффициента замкнутой системы, обычно измеряется в децибелах. Показывает, насколько сильно система колеблется.
Влияние: Высокий резонансный пик указывает на сильные колебания и перерегулирование.
5. Резонансная частота: 3.30 рад/с
Описание: Частота, на которой происходит максимальное колебание замкнутой системы.
Влияние: Эта частота показывает, где система наиболее чувствительна к возмущениям.
6. Полоса пропускания: 5.04 рад/с
Описание: Полоса пропускания — это диапазон частот, в котором амплитудный коэффициент замкнутой системы остается выше определенного порога (обычно 0.707 или -3 дБ).
Влияние: Широкая полоса пропускания позволяет системе реагировать на более широкий диапазон частот, но может увеличить чувствительность к шуму.
7. Эмпирическое время регулирования (5% критерий): 0.50 с
Описание: Время, за которое выходная величина системы остается в пределах 5% от установившегося значения, измеренное экспериментально.
Влияние: Быстрое время регулирования желательно для систем, требующих быстрого ответа.
8. Теоретическое время регулирования: 15.00 с
Описание: Расчетное время регулирования по теоретическим моделям.
Влияние: Различие между теоретическим и эмпирическим временем регулирования может указывать на неточности в модели или влияние внешних факторов.
9. Период колебаний: 1.90 с
Описание: Время, за которое система совершает один полный цикл колебаний.
Влияние: Более короткий период колебаний указывает на более быстрый переходный процесс.
10. Теоретическое перерегулирование: 82.71%
Описание: Расчетное значение перерегулирования по теоретическим моделям.
Влияние: Высокое перерегулирование может привести к нестабильности и колебаниям.
11. Теоретическое время переходного процесса (разомкнутой): 15.00 с
Описание: Расчетное время переходного процесса для разомкнутой системы.
Влияние: Это время показывает, как долго система будет колебаться после возмущения в разомкнутом режиме.

In [ ]:
# Параметры передаточной функции
k = 0.5
T1 = 6
T2 = 4
xi = 0.6

# Передаточная функция W_p = (k * p) / ((T1 * p + 1) * (T2^2 * p^2 + 2 * T2 * xi * p + 1))
num = [k, 0]  # k*p
den = [T1 * T2**2, T1 * 2 * T2 * xi, T1 + T2**2, 2 * xi * T2, 1]
system = lti(num, den)

t_max = 500
t = np.linspace(0, t_max, 50000)

# Единичное ступенчатое воздействие
_, y_step = system.step(T=t)

# Определение локальных экстремумов
extrema_times = []
extrema_values = []
for i in range(1, len(t)-1):
    if (y_step[i] > y_step[i-1] and y_step[i] > y_step[i+1]) or \
       (y_step[i] < y_step[i-1] and y_step[i] < y_step[i+1]):
        extrema_times.append(t[i])
        extrema_values.append(y_step[i])

# Расчет периода колебаний по последовательным максимумам или минимумам
periods = []
for i in range(2, len(extrema_times)):
    if (extrema_values[i] * extrema_values[i-2] > 0):  # если экстремумы одного знака
        periods.append(extrema_times[i] - extrema_times[i-2])
T_osc = np.mean(periods) if periods else 0

# Расчет теоретического периода колебаний
omega_d = 1 / T2 * np.sqrt(1 - xi**2)
T_osc_theory = 2 * np.pi / omega_d

# Расчет декремента затухания
decay_ratios = []
for i in range(2, len(extrema_values)):
    if (extrema_values[i] * extrema_values[i-2] > 0):
        decay_ratios.append(abs(extrema_values[i] / extrema_values[i-2]))
decay_ratio = np.mean(decay_ratios) if decay_ratios else 1

# Расчет перерегулирования
max_value = np.max(y_step)
min_value = np.min(y_step)
steady_state = y_step[-1]
overshoot = (max_value - steady_state) / steady_state * 100 if steady_state != 0 else 0

# Определение времени регулирования
threshold = 0.05 * max(abs(max_value), abs(min_value))
in_band = np.where(np.abs(y_step) <= threshold)[0]
settling_time = t[in_band[0]] if len(in_band) > 0 else None

plt.figure(figsize=(12, 8))
plt.plot(t, y_step, 'b-', linewidth=1.5, label='Переходная характеристика')
plt.plot(extrema_times, extrema_values, 'ro', markersize=4, label='Экстремумы')

plt.axhline(y=threshold, color='green', linestyle='--', label=f'Уровень 5% ({threshold:.4f})')
plt.axhline(y=-threshold, color='green', linestyle='--')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)

plt.plot(extrema_times[0], extrema_values[0], 'mo', markersize=6,
         label=f'Первый максимум: t={extrema_times[0]:.2f}с, y={extrema_values[0]:.4f}')

# Отображение периода колебаний
if len(extrema_times) >= 3:
    plt.annotate(f'T≈{T_osc:.2f}с',
                 xy=(extrema_times[0], extrema_values[0]),
                 xytext=(extrema_times[0]+T_osc/4, extrema_values[0]*1.2),
                 arrowprops=dict(arrowstyle="->"))
    plt.plot([extrema_times[0], extrema_times[2]],
             [extrema_values[0]*0.8, extrema_values[0]*0.8], 'g-')

plt.grid(True)
plt.xlabel('Время (с)')
plt.ylabel('Амплитуда')
plt.title('Переходная характеристика системы с показателями качества')
plt.legend()

# Частотный анализ
omega = np.logspace(-2, 2, 1000)
w, h = freqresp(system, omega)
mag = np.abs(h)
phase = np.angle(h, deg=True)

# Построение АЧХ и ФЧХ
fig, ax1 = plt.subplots(figsize=(12, 8))
ax1.set_xlabel('Частота (рад/с)')
ax1.set_ylabel('Амплитуда', color='blue')
ax1.semilogx(w, mag, 'blue', linewidth=2)
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, which="both", ls="--")

ax2 = ax1.twinx()
ax2.set_ylabel('Фаза (градусы)', color='red')
ax2.semilogx(w, phase, 'red', linewidth=2)
ax2.tick_params(axis='y', labelcolor='red')

# Нахождение резонансной частоты для ACX
resonance_idx = np.argmax(mag)
resonance_freq = w[resonance_idx]
resonance_mag = mag[resonance_idx]

# Маркировка резонансной частоты
ax1.plot(resonance_freq, resonance_mag, 'o', markersize=6, color='darkblue')
ax1.annotate(f'ω_р={resonance_freq:.3f}, M={resonance_mag:.3f}',
             xy=(resonance_freq, resonance_mag),
             xytext=(resonance_freq*1.5, resonance_mag*0.9),
             arrowprops=dict(arrowstyle="->"))

plt.title('Частотные характеристики системы (АЧХ и ФЧХ)')
plt.tight_layout()

# Вывод прямых показателей качества
print("Прямые показатели качества системы:")
print(f"Максимальное значение: {max_value:.4f}")
print(f"Минимальное значение: {min_value:.4f}")
print(f"Перерегулирование: {overshoot:.2f}%")
print(f"Время первого максимума: {extrema_times[0]:.2f} с")
print(f"Период колебаний (фактический): {T_osc:.2f} с")
print(f"Период колебаний (теоретический): {T_osc_theory:.2f} с")
print(f"Коэффициент затухания: {decay_ratio:.4f}")
print(f"Время регулирования (5%): {settling_time:.2f} с" if settling_time else "Время регулирования не достигнуто")
print(f"Показатель колебательности (M): {resonance_mag:.3f}")
print(f"Резонансная частота: {resonance_freq:.3f} рад/с")

plt.show()


Анализируя ваш код и результаты, я вижу важный момент: система действительно не затухает. Это не ошибка в коде, а характеристика самой передаточной функции.

Передаточная функция системы W_p = (k * p) / ((T1 * p + 1) * (T2^2 * p^2 + 2 * T2 * xi * p + 1)) с заданными параметрами (k = 0.5, T1 = 6, T2 = 4, xi = 0.6) представляет собой систему с очень слабым затуханием, что и видно на первом графике.

Причины отсутствия затухания:

Наличие множителя p в числителе делает систему дифференцирующим звеном (или системой типа 1). Такие системы не имеют конечного установившегося значения при ступенчатом воздействии.

Коэффициент затухания 0.9994 (практически равен 1) указывает на то, что амплитуда колебаний почти не уменьшается от периода к периоду.

Экстремально высокий показатель колебательности M = 13.182 подтверждает, что система находится на грани устойчивости.

Резкий резонансный пик на частотной характеристике (на частоте 0.407 рад/с) также указывает на слабое демпфирование колебаний.

Отрицательное значение перерегулирования (-2079.59%) возникает из-за неправильного расчета при близком к нулю или отрицательном установившемся значении. В системах типа 1 с производной в числителе понятие "перерегулирование" обычно рассматривается иначе.

Для корректного анализа такой системы следует:

Рассматривать её как колебательную систему без устойчивого состояния

Оценивать амплитуду колебаний в абсолютных величинах

Использовать частотные характеристики для оценки качества

Система действительно не достигает установившегося режима даже за 5000 секунд, что является её характерной особенностью, а не ошибкой расчёта.